In [1]:
#pip install xelatex
#pip install dash==2.0.0
#pip install dash-extensions
#pip install colorlover
#pip install dash_bootstrap_components
#pip install arch
#pip install plotly==5.4.0

import plotly #Plotly.py 4.7 + Python 3.7 for performance boost 
import plotly.graph_objects as go
import plotly.express as px
import datetime
from datetime import datetime as dt 
from datetime import timedelta

import dash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import dash_table
from dash import html
import plotly
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import math
import time
from time import perf_counter
import datetime
import random
import colorlover
import statsmodels.api as sm
from arch import arch_model
from scipy.optimize import fmin, minimize
import scipy.stats
from scipy.stats import t
from scipy.stats import norm
from math import inf

import warnings
warnings.filterwarnings('ignore')

'''
Licenses:

    DCC Model computations:
    The MIT License (MIT)
    Copyright (c) 2021 tinoproductions

    Dash:
    The MIT License (MIT)
    Copyright (c) 2016-2018 Plotly, Inc

    Bootstrap:
    The MIT License (MIT)
    Copyright (c) 2011-2018 Twitter, Inc.
    Copyright (c) 2011-2018 The Bootstrap Authors
'''

'''Start app'''

alpha=0.05
#right tailed test
z_score=norm.ppf(1-alpha)
#initialmargin
#Fed requirement is 50% 
#exchange requirement is 30%
#maintenance margin not respected
daysofhistoricaldata=5
interval=20000

'''Dow Jones 30 tickers-as of 10-21'''
tickers=['UNH','GS','HD','MSFT','CRM','MCD','V','HON','BA','AMGN','CAT','MMM','DIS','AXP','JPM','JNJ','NKE','TRV','PG','AAPL','IBM','WMT','CVX','MRK','DOW','CSCO',
'KO','INTC','VZ','WBA']
#tickers=['AAPL','GS','HD']

#Empty Position and orderhistory table
orderhistory = pd.DataFrame(columns=['Ticker', 'Price', 'Size','Notional','Buy-in','Trade Profit']).set_index(['Ticker'])

#for plotly graph
rangebreaks=[
                dict(bounds=[16, 9.5], pattern="hour"), #hide hours outside of 16pm-9m
                dict(bounds=["sat", "mon"]), #hide weekends
                dict(values=["2019-01-01","2019-01-21","2019-02-18","2019-04-19","2019-05-27",
                             "2019-07-04","2019-09-02","2019-11-28","2019-12-25","2020-01-01",
                             "2020-01-20","2020-02-17","2020-04-10","2020-05-25","2020-07-03",
                             "2020-09-07","2020-11-26","2020-12-25","2021-01-01","2021-01-18",
                             "2021-02-15","2021-04-02","2021-05-31","2021-07-05","2021-09-06",
                             "2021-11-25","2021-12-24"])  # stock market holidays
            ]


df = pd.DataFrame(columns=['Ticker', 'Price', 'Size','Notional','Buy-in','Trade Profit','Inv. Val.','P/L','Div. Effect'])

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)

app.title = 'RM Tool'

#frontend dash_datatable styling elements
def make_tooltip(text,ID):
    return dbc.Tooltip(
        f"{text}",
        target=ID,
        placement="right",
        style = {
        'maxWidth': 1000,
        'width': 1000
        }
    )

#------------------------------Tab1-----------------------------
#instructions
modal = html.Div(
    [
        html.Div(
            [
                dbc.Button("Instructions", id="open-body-scroll", className="me-1", n_clicks=0),
            ]   
        ),
        dbc.Modal(
            [
                dbc.ModalHeader(dbc.ModalTitle("Instructions")),
                dbc.ModalBody([
                html.Div("Welcome to this demonstration of the DCC MGARCH model based intraday risk management tool."),
                html.Br(),        
                html.Div("The Application consists of two tabs. The first tab is designed to monitor risk on a simulated random portfolio of trading positions that accumulate over the course of the trading day. Parameters for the simulation can be altered before starting the simulation."),
                html.Br(),
                html.Div("Once you have made yourself familiar with how the application behaves, the second tab enables evaluating risk up to the end of the trading day if the portfolio remains unchanged. This includes a historical scenario of extreme historical returns as well as a Monte Carlo simulation. "),
                ]),
                dbc.ModalFooter(
                    dbc.Button(
                        "Close",
                        id="close-body-scroll",
                        className="ms-auto",
                        n_clicks=0,
                    )
                ),
            ],
            size="lg",
            id="modal-body-scroll",
            scrollable=True,
            is_open=False,
        ),
                dbc.Modal(
            [
                dbc.ModalHeader(dbc.ModalTitle("Risk")),
                dbc.ModalBody('LOREM2'),
                dbc.ModalFooter(
                    dbc.Button(
                        "Close",
                        id="close-body-scroll2",
                        className="ms-auto",
                        n_clicks=0,
                    )
                ),
            ],
            size="lg",
            id="modal-body-scroll2",
            scrollable=True,
            is_open=False,
        ),
    ]
)

simparameters = dbc.Card(
        [
        html.H3("Parameters"),
        html.Div(
            [
                dbc.Label("Bid-Ask Spread (%) ",id='tooltip-bid_ask'),
                dbc.Input(id='bid_ask',value=0.1, type="number", min=0, max=0.2, step=0.01, className="form-control"),
                make_tooltip("All trades are simulated on market prices +- 1/2 of the bid-ask spread",'tooltip-bid_ask')
            ]
        ),
        html.Div(
            [
                dbc.Label("Maximum Ordersize ",id='tooltip-max_order_size'),
                dbc.Input(id='max_order_size', value=50000, type="number", min=5000, max=100000, step=5000, className="form-control"),
                make_tooltip("Ordersizes are normally distributed between (-/+) maximum ordersize (short/long) with a mean of 0",'tooltip-max_order_size')
            ]
        ),
        html.Div(
            [
                dbc.Label("Orders Per Iterration ",id='tooltip-order_num'),
                dcc.RangeSlider(id='order_num',
                    marks={i: '{}'.format(i) for i in range(5, 16)},
                    min=5,
                    max=15,
                    value=[10, 15]
                ),
                make_tooltip("Range of random orders simulated upon each iterration",'tooltip-order_num'), 
        html.Div(
            [
                dbc.Label("Margin Requirement On Short Positions (%)",id='tooltip-margin'),
                dbc.Input(id='margin',value=80, type="number", min=0, max=150, step=10, className="form-control"),
                make_tooltip("Short positions typically require a margin account. The FED minimum requirement is 50%. Exchanges typically require 30%. Therefore 80% is set as a default. Brokerage requirements vary but total requirements can add up to 150%. For simplicity, the initial margin is also the maintenance margin and updates as such throughout the lifetime the position is held",'tooltip-margin'),
            ]
        ),
        dbc.Label("Date",id='tooltip-date-picker'),
        html.Div(
            [
        dcc.DatePickerSingle(
        id='date-picker',
        calendar_orientation='horizontal',
        day_size=39,
        first_day_of_week=1,
        reopen_calendar_on_clear=True,
        number_of_months_shown=1, 
        min_date_allowed=dt(2020, 10, 15),
        max_date_allowed=dt(2021, 10, 15), 
        date=dt(2020, 10, 27),
        initial_visible_month=dt(2020, 10, 1),
        display_format='DD/MM/Y',
        month_format='MMMM, YYYY', 
        ),
        make_tooltip("Sample choice extends back one year. If the chosen date is a non-trading day or a day with shorter hours, the next full day from the sample is chosen.",'tooltip-date-picker'),
            ]),
        html.Hr(),        
        html.Div(
            [
                dbc.Button("Start",id='start', color="success", disabled=False, className="me-1", n_clicks_timestamp='0'),
                dbc.Button("Stop",id='stop', color="danger", disabled=True, className="me-1", n_clicks_timestamp='0'),
                dbc.Button("Reset",id='reset', color="secondary", disabled=True,className="me-1", n_clicks_timestamp='0'),
                        ]   
                    ), 
                ]
        )
        ], body=True,
)

simstats = dbc.Card(
        [
        dbc.Row(
            [
                dbc.Col(html.H3("Simulation Status")),
                dbc.Col(html.Div(id='current-time',className="float-end"), md=6),
            ],
            className="mt-3",
        ),    
        dbc.Label("Status"),
        dbc.Badge("Idle", id="status", className="ms-1"),

        dbc.Progress(id="progress", striped=True, animated=True), 
        html.Br(),
        html.Br(),
        html.Div(
                [
                
                dcc.Loading( id="loading",type="default",children=html.Div(id="loading-output"))
                    ]   
                ),
        
        
        
        html.Br(),
        html.H5("Computing Time per Iterration:"),
        html.Div(
            [
                dbc.Label("Param. Opt.:",id="paramopttime"),
                make_tooltip("Time required for parameter optimization of scipy's SLSQP algorithm. Time will decrease after the first optimization as previous parameters are reused",'paramopttime')
            ]
        ),
        html.Div(
            [
                dbc.Label("Risk:",id="risksimtime"),
                make_tooltip("Time required for both parameter optimization and risk related figures",'risksimtime')
            ]
        ),    
        html.Div(
            [
                dbc.Label("Total:",id="totalsimtime"),
                make_tooltip("Time required for both risk and formatting",'totalsimtime')
            ]
        ),   
        ], body=True,
)

#additionalstyle
style_data_conditional_positiontable = [
    {'if': {'column_id': 'Buy-in',},
                    'display': 'None',},
    
    {           
        'if': {
                        'column_id': 'Ticker'
                    },
                    'fontWeight':'bold','backgroundColor': 'rgb(248,248,248)'
    },
    {
        "if": {"state": "active"},
        "backgroundColor": "rgba(150, 180, 225, 0.2)",
        "border": "1px solid blue",
    },
    {
        "if": {"state": "selected"},
        "backgroundColor": "rgba(0, 116, 217, .03)",
        "border": "1px solid blue",
    },
]

positiontable = dbc.Card(
    [
        html.H3("Position Table"),
        
        html.Div(
            [
                dash_table.DataTable(
        id='positiontable',          
        columns=[
            {'name': i, 'id': i, 'selectable': True, } for i in df.columns
        ],
        data=df.to_dict('records'),
        sort_action="native",
        sort_mode="multi",
        selected_columns=[],
        selected_rows=[],
        style_data_conditional=style_data_conditional_positiontable,
        page_action="native",
        style_header_conditional=[
                {'if': {'column_id': 'Buy-in',},
                    'display': 'None',}],
        style_header={'fontWeight':'bold','fontSize':15},
        style_cell={'fontSize':15, 'font-family':'Helvetica', 'textAlign': 'center'},
        style_table={'overflowX': 'auto', 'overflowY': 'auto'}
    ),
                
            ]

        ),
    ],
    body=True,
)

VaR = dbc.Card(
    [
        dbc.Checklist(
            options=[
                {"label": "Incl. Fl. Profits", "value": 1, "disabled": True},
            ],
            value=[0],
            id="boolean-flprofits",
            switch=True,
        ),
        
        html.H3("Value at Risk"),
        
        html.Div([
        html.P(id="VaR-noexpret")
        ],style= {'display': 'block'}),
        html.Div([
        html.P(id="VaR-expret")
        ]),
        
        html.H3("Expected Shortfall"),
        html.Div([
        html.P(id="Es-noexpret")
        ],style= {'display': 'block'}),
        html.Div([
        html.P(id="Es-expret")
        ])
    ],
    body=True,
)

correlationmatrix = dbc.Card(
    [
        
        html.H3("Dynamic Conditional Correlation Matrix"),
        
        html.Div(id='dcc_matrix'),
        html.Div(id='dcc_correlations'),
        
    ],
    body=True,
)

modelparameters=dbc.Card(
         [
            html.H3("DCC Model Parameters"),

            html.Div([
            dbc.Label("a:",id="model-parameter-a"),
            make_tooltip("dcc a captures the short-run volatility impact. Values around 0.01 are usual in quiet markets. Values around 0.05 typically reflect stressed markets",'model-parameter-a'),
            ]),
            html.Div([
                dbc.Label("b:",id="model-parameter-b"),
                make_tooltip("dcc b captures the modelled volatility of t-1. This reflects the persistence of the conditional correlation process",'model-parameter-b'),
            ]),
            html.Div([
                dbc.Label("1-a-b:",id="model-parameter-1-a-b"),
                make_tooltip("1-a-b is the weighting of the long run matrix of correlations and captures long run mean-reversion of the model",'model-parameter-1-a-b'),
            ]),
            html.Div(id='univariateparams')
     ],
     body=True,
    style={"width": "70rem"},
)

simulation_layout = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(simparameters, md=6),
                dbc.Col(simstats, md=6),
            ],
            className="mt-3",
        ),
        
        html.Hr(),
        dbc.Button(
            "VaR & ES",
            color="primary",
            id="toggle-var",
            className="me-1",
            size="sm",
            n_clicks=0,
        ),
        dbc.Button(
            "DCC Model Parameters",
            color="primary",
            id="toggle-dccmodel",
            className="me-1",
            size="sm",
            n_clicks=0,
        ),
        dbc.Button(
            "Position Table",
            color="primary",
            id="toggle-positiontable",
            className="me-1",
            size="sm",
            n_clicks=0,
        ),
        dbc.Button(
            "DCC Matrix",
            color="primary",
            id="toggle-dccmatrix",
            className="me-1",
            size="sm",
            n_clicks=0,
        ),
        dbc.Button("Toggle All", color="primary", id="all", size="sm", n_clicks=0),       
        dbc.Row(
            [
                dbc.Col(
                    dbc.Collapse(
                        VaR,
                        id="var-collapse",
                        is_open=True,
                    )
                ),
                dbc.Col(
                    dbc.Collapse(
                        modelparameters,
                        id="dccmodel-collapse",
                        is_open=True,
                    )
                ),
            ],
            className="mt-3",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dbc.Collapse(
                        positiontable,
                        id="positiontable-collapse",
                        is_open=True,
                    )
                ),
            ],
            className="mt-3",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dbc.Collapse(
                        correlationmatrix,
                        id="dccmatrix-collapse",
                        is_open=True,
                    )
                ),
            ],
            className="mt-3",
        ),
        dcc.Interval(             
                id='interval-component',             
                interval=interval, # in milliseconds             
                n_intervals=0,
                disabled=True
        )
    ]
)

#-----------------------tab2------------------------------------------------------------------------

loadpositiontable = dbc.Card(
        [
        dbc.Row(
            [
                dbc.Col(dbc.Button("Load Positions",id='load', color="success", disabled=False, className="me-1", n_clicks_timestamp='0')),
                dbc.Col(html.Div(id='current-time2',className="float-end"), md=6),
            ],
            className="mt-3",
        ),
        html.P(" "),
        html.Div(
            [      
                html.Div(id='positiontable2'),
            ]   
                    ), 
                ],body=True       
        )

historicalevent=dbc.Card(
        [
        html.Div(
            [
            html.H3("1: Historical Scenario"),
            dbc.DropdownMenu(
            [
            dbc.DropdownMenuItem("Black Monday   1987-10-19   -22.61%",id='option1',style={'backgroundColor': '#f46e6e'}),
            dbc.DropdownMenuItem('Stock Market Crash   2020-03-16   -12.93%',id='option2',style={'backgroundColor': '#f46e6e'}),
            dbc.DropdownMenuItem("Wall Street Crash   1929-10-28   -12.82%",id='option3',style={'backgroundColor': '#f46e6e'}),
            dbc.DropdownMenuItem(divider=True),
            dbc.DropdownMenuItem("Emergency Banking Act   1933-03-15   +15.34%",id='option4',style={'backgroundColor': '#9cf986'}),
            dbc.DropdownMenuItem('Financial Bill By Hoover   1931-10-06   +14.87%',id='option5',style={'backgroundColor': '#9cf986'}),
            dbc.DropdownMenuItem("Pre Crash Rise   1929-10-06   +12.34%",id='option6',style={'backgroundColor': '#9cf986'}),    
            dbc.DropdownMenuItem(divider=True),
            html.P(
                "*In descending order",
                className="text-muted px-4 mt-4",style={'fontSize':12},
            ),
        ],
        label="Select Scenario",
        id='dropdownscenario',
        disabled=True,
        )
            ]
        ),
        html.Br(),
        html.Div(id='historicloss')
        ],body=True
)

montecarlo=dbc.Card(
        [
        html.Div(
            [
            html.H3("2: Monte Carlo Simulation"),
            html.Div(
            [
                dbc.Label("Simulations"),
                dbc.Input(id='mc-num',value=5000, type="number", min=1000, max=20000, step=1000,disabled=True, className="form-control"),
            ],
            ),
            html.Hr(),
            
            html.Div(
            [
                dbc.Button("Run",id='mc-run', color="success", disabled=True, className="me-1", n_clicks_timestamp='0'),
                        ]   
                    ),
            html.P(" "),
            html.P(" "),
            dcc.Loading(id = "loading3", 
                children=[
                    html.Div(" ",
                    id="mc"
                    ),
                ], type="default")
                ]),
                ],body=True
        )

stresstest_layout = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(
                        loadpositiontable
                ),
            ],
            className="mt-3",
        ),
        dbc.Row(
            [
                dbc.Col(
                        historicalevent
                ),
            ],
            className="mt-3",
        ),
        dbc.Row(
            [
                dbc.Col(
                        montecarlo
                ),
            ],
            className="mt-3",
        ),
    ]
)

#------------------------------Applayout------------------------------            

 
app_tabs = html.Div(
    [
        dbc.Tabs(
            [
                dbc.Tab(label="Live Simulation", tab_id="simulation", labelClassName="text-success font-weight-bold", activeLabelClassName="text-secondary"),
                dbc.Tab(label="Adverse Scenario Analysis", tab_id="stresstest", labelClassName="text-success font-weight-bold", activeLabelClassName="text-secondary"),
            ],
            id="tabs",
            active_tab="simulation",
        ),
        #tab1 storage
        dcc.Store(id='store-theta'),
        dcc.Store(id='store-Ht'),
        dcc.Store(id='store-rets'),
        dcc.Store(id='store-margin'),
        dcc.Store(id='Var-expret-value'),
        dcc.Store(id='Es-expret-value'),
        dcc.Store(id='timestamplist'),
        dcc.Store(id='store-correlations'),
        dcc.Store(id='store-extractsamplebool'),
        #tab1 storage for tab2
        dcc.Store(id='store-positiontable'),
        dcc.Store(id='store-positiontable-style_data_conditional'),
        dcc.Store(id='store-exposure'),
        dcc.Store(id="store-notionalsmargin"),
        dcc.Store(id="store-ddofs")
        
    ], className="mt-3"
)

app.layout = dbc.Container([
        html.H1("DCC Garch Tool"),
        html.Hr(style={'border': '100%', 'height': '2px'}),
        #instructions
        modal,
        html.Hr(style={'border': '100%', 'height': '2px'}),
        dbc.Row(dbc.Col(app_tabs, width=12), className="mb-3"),
        html.Div(id='content', children=[])

],fluid=True,)

#------------------------------Header&Tab Callbacks---------------------
#instruction manual    
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open

app.callback(
    Output("modal-body-scroll", "is_open"),
    [
        Input("open-body-scroll", "n_clicks"),
        Input("close-body-scroll", "n_clicks"),
    ],
    [State("modal-body-scroll", "is_open")],
)(toggle_modal) 


#tab callback
@app.callback(
    Output("content", "children"),
    [Input("tabs", "active_tab")]
)

def switch_tab(tab_chosen):
    if tab_chosen == "simulation":
        return simulation_layout
    elif tab_chosen == "stresstest":
        return stresstest_layout
    return html.P("This shouldn't be displayed for now...") 

#------------------------------Stresstest Tab Callbacks-----------------------    
    
    
#importing positions
@app.callback(
    [
    Output("current-time2",'children'),
    Output("positiontable2", "children"),
    Output("load", "disabled"),
    Output("load", "size"),
    Output("load", "outline"),
    Output("store-exposure", "data"),
    Output("store-notionalsmargin", "data"),    
    Output('option1','disabled'),
    Output('option2','disabled'),
    Output('option3','disabled'),
    Output('option4','disabled'),
    Output('option5','disabled'),
    Output('option6','disabled'),
    Output('dropdownscenario','disabled'),
    Output('mc-run','disabled'), 
    Output('mc-num','disabled'),
    ],
    [Input("load", "n_clicks")],
    [State("store-positiontable", "data"),
    State("store-positiontable-style_data_conditional", "data"),
    State("load", "disabled"),
    State("store-extractsamplebool", "data"),#contains date
    State("store-margin", "data") 
    ],prevent_initial_call=True
)

def loadpositions(#Input
                n_load,
                #States
                store_positiontabledata,
                store_positiontablestyle, 
                loadbuttonstate, 
                currentdate,
                marginrequirement):
    
    
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]
    
    if n_load==None:
        raise dash.exceptions.PreventUpdate
    n_load=int(n_load)
    if n_load==0 or None:
        raise dash.exceptions.PreventUpdate
    
    if store_positiontabledata==None:
        nopositionalert=dbc.Alert(
                        [
                        html.H4('No Positions' , className="alert-heading"),
                        html.Hr(),
                        html.P("Please run simulation to accumulate a portfolio."),
                        ],color="danger",dismissable=True,
                )
        return "",html.Div([nopositionalert]), False,"", False, 0, False, False, False, False, False, False, True,True,True, False
    else:
        df_positions=pd.DataFrame(store_positiontabledata).set_index('Ticker')
        df_positions['longshort'] = np.where(df_positions['Size']> 0, True, False)
        df_positions['Notional_w_margin']=df_positions['Notional'][df_positions['longshort'] == False]*(1+marginrequirement)
        df_positions['Notional_w_margin']=np.where(df_positions['Notional_w_margin'].isnull(), df_positions['Notional'], df_positions['Notional_w_margin'])
        exposure=np.round((df_positions['Notional_w_margin'].to_numpy().sum()),2)
        notionals_margin_df=df_positions['Notional_w_margin']
        df_positions.drop(['Notional_w_margin', 'longshort'], axis=1, inplace=True)

        if exposure >=0:
            status='Long'
        else:
            status='Short'
        
        if marginrequirement==0:
            marginalert=''
        else:
            marginalert='*Including Margin On Short Positions'

        exposurealert=dbc.Alert(
                            [
                            html.H4('Exposure' , className="alert-heading"),
                            html.Hr(),
                            html.P("%d %s" % (exposure,status)),
                            html.P(marginalert,style={'fontSize':12})
                            ],color="primary",dismissable=True,
                    )
        
        positiontable2_htmldiv = html.Div(
                                    [ 
                                    html.Br(),
                                    dash_table.DataTable(
                                    id='positiontable2',          
                                    columns=[
                                        {'name': i, 'id': i, 'selectable': True, } for i in df.columns
                                    ],
                                    data=store_positiontabledata,
                                    sort_action="native",
                                    sort_mode="multi",
                                    selected_columns=[],
                                    selected_rows=[],
                                    style_data_conditional=store_positiontablestyle,
                                    page_action="native",
                                    style_header_conditional=[
                                            {'if': {'column_id': 'Buy-in',},
                                                'display': 'None',}],
                                    style_header={'fontWeight':'bold','fontSize':15},
                                    style_cell={'fontSize':15, 'font-family':'Helvetica', 'textAlign': 'center'},
                                    style_table={'overflowX': 'auto', 'overflowY': 'auto'}
                                    ),
                                    html.Br(),
                                    exposurealert
                                    ])

        if exposure >=0:
            return currentdate,positiontable2_htmldiv, True,"sm", True, exposure,notionals_margin_df, False, False, False, True, True, True, False,False,False
        else:
            return currentdate,positiontable2_htmldiv, True,"sm", True, exposure,notionals_margin_df, True, True, True, False, False, False, False,False,False

#historical simulation   
@app.callback(
    [
    Output("historicloss", "children"),
    Output("dropdownscenario",'label'),
    ],
    [
        Input("option1", "n_clicks"),
        Input("option2", "n_clicks"),
        Input("option3", "n_clicks"),
        Input("option4", "n_clicks"),
        Input("option5", "n_clicks"),
        Input("option6", "n_clicks"),
    ],
    [
        State('store-positiontable','data'),
        State('store-exposure','data'),
        State('store-margin','data'),
        State("store-notionalsmargin",'data'),
    ],prevent_initial_call=True
)
def historicloss(opt1,opt2,opt3,opt4,opt5,op6,data,exposure,marginrequirement,notionals_margin):
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]

    df_positions=pd.DataFrame(data).set_index('Ticker')
    floatingprofit=df_positions['P/L'].sum()

    historicevents_df=pd.DataFrame({'date':  ['1987-10-19', '2020-03-16','1929-10-28', '1933-03-15','1931-10-06','1929-10-06'],
        'event': ['Black Monday 1987', '2020 Stock Market Crash','Wall Street Crash of 1929','Emergency Banking Act 1933','Financial Bill By Hoover 1931','1929 Pre Crash Rise'],
        'eventtext': ['Black Monday 1987', 'the 2020 Stock Market Crash','the Wall Street Crash of 1929','Emergency Banking Act 1933','the day the Financial Bill By Hoover was passed in 1931','the 1929 Pre Crash Rise'],
        'change': [-0.2261 , -0.1293 , -0.1282 , 0.1534 , 0.1487 , 0.1234],
        'avg. 5min ret': [-0.00332 , -0.0018 , -0.00178 , 0.001855 , 0.001802 , 0.001512]                        
        })
    
    option = int(input_id[-1])-1
    
    ret=historicevents_df.at[option, 'avg. 5min ret']
    ttoendofday=len(pd.read_csv("sample_sim.csv").index)
    
    notionals_margin_df=pd.DataFrame(notionals_margin)
    notionals_margin_df['historicloss']=notionals_margin_df*(1+ret)**ttoendofday
    
    historicvalue=np.round((notionals_margin_df['historicloss'].to_numpy().sum()),2)
    historicloss=np.sqrt((exposure-historicvalue-floatingprofit)**2)
    
    historiclossalert_htmldiv=html.Div(
                    [
                            dbc.Alert(
                        [
                        html.H4("The expected end of day loss is %d" % historicloss, className="alert-heading"),
                        html.Hr(),
                        html.P(
                            "Following the assumption that the portfolio's positions follow the average 5 minute return trajectory of %s until the end of the day including floating profits." % historicevents_df.at[option, 'eventtext'],
                            className="mb-0",
                        ),    

                        ],color="primary",dismissable=True,
                    )
                    ])
    
    event=historicevents_df.at[option, 'event']    
    return historiclossalert_htmldiv,event

#Monte Carlo
@app.callback(
    Output("mc", "children"),
    Input('mc-run', "n_clicks"),
    [
    State('mc-num','value'),
    State('store-exposure','data'),
    State('store-rets','data'),
    State('store-Ht','data'),
    State('store-positiontable','data'),
    State('store-margin','data'), 
    State('store-ddofs','data'), 
    ],prevent_initial_call=True
)
def MonteCarlo(mc_run,mc_num,exposure,rets,covMatrix,data,marginrequirement, ddofs):

    rets=pd.DataFrame(rets)
      
    meanrets=(rets.mean(axis=0)).to_numpy()
    meanrets=np.reshape(meanrets, (len(meanrets),1))
    covMatrix = np.array(covMatrix)
    
    sample_sim=pd.read_csv("sample_sim.csv").set_index('date')
    ttoendofday=len(sample_sim.index)

    df_positions=pd.DataFrame(data).set_index('Ticker')
    nstocks=df_positions.loc[:,'Size'].values
    bool_ind = nstocks < 0
    #increasing short position by marginrequirement
    nstocks = np.where(bool_ind ==True, nstocks*(1+marginrequirement), nstocks)
    
    lastprice=df_positions.loc[:,'Price'].values
    
    #determine weighted average of univariate ddofs form univariate garch fit
    ddofs=pd.DataFrame(ddofs)
    weights_df=(df_positions['Notional'].abs())/(df_positions['Notional'].abs().sum())
    weights=weights_df.values
    ddofs=np.delete(ddofs.values[0],[0])
    #used subsequently for random draws
    ddof_weightedavg=np.round(np.sum(np.multiply(weights,ddofs)),3)
    
    portval=np.sum(np.multiply(lastprice,nstocks))
    portval=np.repeat(portval, mc_num)
    
    # Monte Carlo Method
    cholesky = np.linalg.cholesky(covMatrix)
    counter=0
    for i in range(0, ttoendofday-1):
        #draw random normal returns
        draw=np.random.standard_t(ddof_weightedavg, mc_num) #averaage univariate ddof from portfolio
        for x in nstocks[:-1]:#less one as first draw already completed
            draw=np.vstack((draw, np.random.standard_t(ddof_weightedavg, mc_num)))
        #correlated random returns
        corrdraw=np.matmul(cholesky, draw)
        

        #Add expected returns
        corrdraw_plusexpret=np.add(meanrets, corrdraw)
        corrdraw_plusexpret=1+corrdraw_plusexpret #for price multiplication

        #price development of individual stocks
        if counter==0:
            stockpricechanges=(corrdraw_plusexpret.T * lastprice).T
            lastprice=stockpricechanges

        else:
            stockpricechanges=np.multiply(lastprice,corrdraw_plusexpret)
            lastprice=stockpricechanges

        stockpricenotionals=(stockpricechanges.T * nstocks).T

        portval=np.vstack((portval,stockpricenotionals.sum(axis=0)))

        counter+=1

    portvalprocess = pd.DataFrame(portval)  
        
    max_df = portvalprocess.max(axis=1)
    min_df = portvalprocess.min(axis=1)

    portvalprocess_transposed=portvalprocess.T
    confidence_transposed=portvalprocess_transposed.quantile( [alpha, .5, 1-alpha], axis = 0)
    confidence=confidence_transposed.T
    
    fig = go.Figure()

    Colors = colorlover.scales[str(5)]['seq']['Blues']    
    #max
    fig.add_trace(go.Scatter(
        x=sample_sim.index,
        y=max_df,
        name='max',
        mode = 'lines',
        line=dict(
                    color=Colors[2],
                    width=3
                ),
        showlegend=False
        ))
    #min
    fig.add_trace(go.Scatter(
        x=sample_sim.index,
        y=min_df,
        name='min',
        mode = 'lines',
        line=dict(
                    color=Colors[2],
                    width=3
                ),
        fill='tonexty',
        fillcolor = Colors[1],
        showlegend=False
        ))
    #mean
    fig.add_trace(go.Scatter(
        x=sample_sim.index,
        y=confidence.iloc[:,1],
        name='mean',
        mode = 'lines',
        line=dict(
                    color=Colors[4],
                    width=3
                ),
        showlegend=False
        ))

    #lowerconfidence
    fig.add_trace(go.Scatter(
        x=sample_sim.index,
        y=confidence.iloc[:,0],
        name='l. %s.'%(1-alpha),
        mode = 'lines',
        line=dict(
                    color=Colors[4],
                    width=3,
                    dash='dot'
                ),
        showlegend=False
        ))
    #upperconfidence
    fig.add_trace(go.Scatter(
        x=sample_sim.index,
        y=confidence.iloc[:,2],
        mode = 'lines',
        name='u. %s.'%(1-alpha),
        line=dict(
                    color=Colors[4],
                    width=3,
                    dash='dot'
                ),
        showlegend=False
        ))

    fig.update_layout(
    font=dict(
    family="Helvetica",
    size=14,
    color="#292b2c"
    ),
    plot_bgcolor="#FFF",  # Sets background color to white
    legend_title="Variables",
    xaxis=dict(
        title="Time",  
        linecolor="#292b2c",  # Sets color of X-axis line
        showgrid=False,  # Removes X-axis grid lines
        fixedrange=True,
    ),
    yaxis=dict(
        title="Value",  
        linecolor="#292b2c",  # Sets color of Y-axis line
        showgrid=False,  # Removes Y-axis grid lines 
        fixedrange=True,
    ))

    fig.update_xaxes(tickfont_size=10,color="#292b2c",linewidth=1.5)
    fig.update_yaxes(tickfont_size=10,color="#292b2c",linewidth=1.5)
    fig.update_layout(hovermode="x")    

    floatingprofit=df_positions['P/L'].sum()

    ValueAtRisk=np.round(np.sqrt(((confidence.iloc[:, 0].iloc[-1])-(confidence.iloc[:, 0].iloc[0]))**2)-floatingprofit,2)
    
    portvalprocess_sorted = portvalprocess.sort_values(by=portvalprocess.index[-1], axis=1)
    terminalvalues=portvalprocess_sorted.iloc[-1].to_numpy()
    taillosses = terminalvalues[(terminalvalues <= confidence.iloc[:, 0].iloc[-1]) ]
    MCES=np.round(np.sqrt((np.average(taillosses)-(confidence.iloc[:, 0].iloc[0]))**2)-floatingprofit,2)

    MChtml=html.Div([dcc.Graph(
                figure=fig,
                config={"displayModeBar": False, "showTips": False,'scrollZoom': True}
                ),
                dbc.Label("Including Fl. Profits"),
                html.H3("Value at Risk"),

                html.Div([
                html.P("%s" % ValueAtRisk,id="VaR-noexpret2")
                ]),
                html.H3("Expected Shortfall"),
                html.P("%s" % MCES,id="Es-noexpret2")
                    ])
                
    return MChtml
    
#------------------------------Simulation Tab Callbacks------------------------------    

# Buttons & Invalid parameter inputs
@app.callback(
    [
    Output("start", "disabled"),
    Output("stop", "disabled"),
    Output("reset", "disabled"),    
    Output('interval-component', 'disabled'),
    Output('start','children'),
    Output('bid_ask','disabled'),
    Output('max_order_size','disabled'),
    Output('order_num','disabled'),
    Output('margin','disabled'),
    Output('date-picker','disabled'),    
    Output('status','children'),
    Output('status','color'),
    Output('bid_ask','invalid'),
    Output('max_order_size','invalid'),
    ],
    [
    Input('start', 'n_clicks_timestamp'),
    Input('stop', 'n_clicks_timestamp'),
    Input('reset', 'n_clicks_timestamp'),
    Input('bid_ask', 'value'),
    Input('max_order_size', 'value'),
    Input('date-picker', 'date'),
    ],
    [
    State('start','disabled'),
    State('stop', 'disabled'),
    State('reset', 'disabled'),
    State('interval-component', 'disabled'),
    State('start','children'),
    State('bid_ask','disabled'),
    State('max_order_size','disabled'),
    State('bid_ask','invalid'),
    State('max_order_size','invalid'),        
    State('order_num','disabled'),
    State('margin','disabled'),
    State('date-picker','disabled'), 
    State('status','children'),
    State('status','color'),
    ],prevent_initial_call=True
)
def click(  #inputs
            start,
            stop,
            reset,
            bidaskvalue,
            maxordervalue,
            dateselection,
            #states
            startdisabled,
            stopdisabled,
            resetdisabled,
            ndisabled,
            startbuttonlabel,
            bid_askdisabled,
            max_order_sizedisabled,
            bid_askinvalid,
            max_order_invalid,
            order_numdisabled,
            margin_disabled,
            datepicker_disabled,
            statuschildren,
            statuscolor
         ):
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]
    

    
    if input_id in 'start':
        return True, False, False, False, "Start", True, True, True,True, True, 'Running', 'success', False, False
    elif input_id in 'stop':
        return False, True, False, True, "Resume", True, True, True,True, True, 'Stopped', 'danger', False, False
    elif input_id in 'reset':
        return False, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', False, False
    
    
    
    elif input_id in 'bid_ask' and bidaskvalue==None:
        return True, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', True, max_order_invalid
    elif input_id in 'max_order_size' and maxordervalue==None:
        return True, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', bid_askinvalid, True   
    elif (input_id in 'bid_ask' and bidaskvalue==None) or (input_id in 'max_order_size' and maxordervalue==None):
        return True, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', True, True 

    
    elif input_id in 'bid_ask' and bidaskvalue!=None:
        return False, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', False, max_order_invalid
    elif input_id in 'max_order_size' and maxordervalue!=None:
        return False, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', bid_askinvalid, False   
    elif (input_id in 'bid_ask' and bidaskvalue!=None) or (input_id in 'max_order_size' and maxordervalue!=None):
        return False, True, True, True, "Start", False, False, False, False, False, 'Idle', 'secondary', False, False 


    elif input_id in 'date-picker' and dateselection==None:
        return True,True,reset,ndisabled,startbuttonlabel,bid_askdisabled,max_order_sizedisabled,max_order_invalid,margin_disabled,datepicker_disabled,'Idle','secondary',bid_askinvalid,max_order_invalid
    elif input_id in 'date-picker' and dateselection!=None:
        return False,stop,reset,ndisabled,startbuttonlabel,bid_askdisabled,max_order_sizedisabled,max_order_invalid,margin_disabled,datepicker_disabled,'Idle','secondary',bid_askinvalid,max_order_invalid
    
#nclickreset
@app.callback(
    [
    Output("start", "n_clicks_timestamp"),
    Output("stop", "n_clicks_timestamp"),    
    ],
    [
    Input('reset', 'n_clicks_timestamp'),
    ],
    [
    State('start', 'n_clicks_timestamp'),
    State('stop', 'n_clicks_timestamp'),
    ],prevent_initial_call=True
)
def click(stop,startn,stopn):
    return 0,0  

#Collapse
@app.callback(
    Output("var-collapse", "is_open"),
    [Input("toggle-var", "n_clicks"), Input("all", "n_clicks")],
    [State("var-collapse", "is_open")],
)
def toggle_VaR(n_VaR, n_all, is_open):
    if n_VaR or n_all:
        return not is_open
    return is_open

@app.callback(
    Output("positiontable-collapse", "is_open"),
    [Input("toggle-positiontable", "n_clicks"), Input("all", "n_clicks")],
    [State("positiontable-collapse", "is_open")],
)
def toggle_positiontable(n_positiontable, n_all, is_open):
    if n_positiontable or n_all:
        return not is_open
    return is_open

@app.callback(
    Output("dccmatrix-collapse", "is_open"),
    [Input("toggle-dccmatrix", "n_clicks"), Input("all", "n_clicks")],
    [State("dccmatrix-collapse", "is_open")],
)
def toggle_dccmatrix(n_dccmatrix, n_all, is_open):
    if n_dccmatrix or n_all:
        return not is_open
    return is_open

@app.callback(
    Output("dccmodel-collapse", "is_open"),
    [Input("toggle-dccmodel", "n_clicks"), Input("all", "n_clicks")],
    [State("dccmodel-collapse", "is_open")],
)
def toggle_dccmodel(n_dccmodel, n_all, is_open):
    if n_dccmodel or n_all:
        return not is_open
    return is_open

#------------------------------
@app.callback(
    Output("graph-collapse", "is_open"),
    [Input("toggle-closegraph", "n_clicks")],
    [State("graph-collapse", "is_open")],
)
def close_correlationgraph(n_closematrix, is_open):
    if n_closematrix:
        return not is_open
    return is_open


@app.callback(
    Output('dcc_correlations', 'children'),
    [Input('inner_matrix_div', 'active_cell')],
    [State('store-correlations', "data"),
    State('inner_matrix_div', "data")],prevent_initial_call=True
)
def open_correlationgraph(dcc_matrix_activecell,data_corr,data_corrmatrix):

    if dcc_matrix_activecell==None:
        raise dash.exceptions.PreventUpdate    

    if type(dcc_matrix_activecell)==None or type(data_corr)==None or type(data_corrmatrix)==None:
        return ""
    else:
        
        dcc_corr=pd.DataFrame(data_corr)
        dcc_corr=dcc_corr.round(3)
        #dcc_corr['date'] = pd.to_datetime(dcc_corr['date'],format= '%Y-%m-%d %H:%M:%S' )
        dcc_corr.set_index(['date'], inplace=True)
        
        corrmatrix=pd.DataFrame(data_corrmatrix)
        correlationpair=corrmatrix.at[dcc_matrix_activecell['row'],""]+'-'+dcc_matrix_activecell['column_id']
        dcc_corr=dcc_corr[correlationpair]
        
        layout = go.Layout(
            font=dict(
            family="Helvetica",
            size=14,
            color="#292b2c"
            ),
            title='<span style="font-size: 20px;font-weight: bold;">%s</span>'%correlationpair,
            plot_bgcolor="#FFF",  # Sets background color to white
            hovermode="x",
            hoverdistance=100, # Distance to show hover label of data point
            spikedistance=1000, # Distance to show spike
            xaxis=dict(
                title='<span style=";">Date</span>',  
                linecolor="#292b2c",  # Sets color of X-axis line
                showgrid=False,  # Removes X-axis grid lines
                showspikes=True, # Show spike line for X-axis
                # Format spike
                spikethickness=4,
                spikedash="dot",
                spikecolor="#999999",
                spikemode="across",
                fixedrange=True,
            ),
            yaxis=dict(
                title='<span style="font-size: 16px;">Dynamic conditional correlation</span>',  
                linecolor="#292b2c",  # Sets color of Y-axis line
                showgrid=False,  # Removes Y-axis grid lines 
                fixedrange=True,
            )
        )
        fig = go.Figure(
            data=go.Scatter(x=dcc_corr.index, y=dcc_corr,line=dict(color="#08519C")),
            layout=layout
        )
        fig.update_xaxes(tickfont_size=10,color="#292b2c",linewidth=1.5)
        fig.update_yaxes(tickfont_size=10,color="#292b2c",linewidth=1.5)
        fig.update_xaxes(
            rangebreaks=rangebreaks)
        fig.update_layout(yaxis_range=[-1,1])
        
        html_corr_div=dbc.Collapse(
            html.Div([
            html.Br(),
            dbc.Row(
            [
                dbc.Col(html.H3("Historic Dynamic Conditional Correlation")),
                dbc.Col(html.Div(
                        [
                        dbc.Button(
                            "X",
                            color="danger",
                            id="toggle-closegraph",
                            size="sm",
                            className="float-end",
                            n_clicks=0
                            ),
                        
                        ]   
                    ),width={"size": 3, "order": "last"})
            ]),
            dbc.Row(
            [
                dbc.Col(dcc.Loading(id = "loading2", 
                children=[html.Div(dcc.Graph(
                id='correlationgraph-all',
                figure=fig,
                config={"displayModeBar": False, "showTips": False,'scrollZoom': True}
                ))], type="default"))
            ])
            ]),
                    id="graph-collapse",
                    is_open=True,
                    )
        return html_corr_div

@app.callback(
    [
    Output("VaR-noexpret", "style"),
    Output("Es-noexpret", "style"),
    Output("VaR-expret", "children"),
    Output("Es-expret", "children"),
    ],
    Input("boolean-flprofits", "value"),
    Input('reset', 'n_clicks_timestamp'),
    [
    State("Var-expret-value", "data"),
    State("Es-expret-value", "data"),
    ],prevent_initial_call=True
)
def toggle_ExpectedReturns(value,resetclick, Var_expret_value,Es_expret_value):
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]  
    
    if value[-1] == 1 and input_id in 'boolean-flprofits':
        return {'display': 'none'},{'display': 'none'}, Var_expret_value, Es_expret_value
    elif value[-1] == 0:
        return {'display': 'block'},{'display': 'block'}, "",""
    elif input_id in 'reset':
        return {'display': 'none'},{'display': 'none'}, "",""

#DCC Computation --- main callback
@app.callback(
    [
    #tab1 output
    Output('store-theta','data'),    
    Output('store-margin','data'),
    Output('store-Ht','data'),      
    Output('store-rets','data'),       
    Output('store-extractsamplebool','data'),
    Output('current-time','children'),
    Output('boolean-flprofits','options'),
    Output("positiontable", "style_data_conditional"),
    Output('store-positiontable-style_data_conditional', "data"),
    Output("loading-output", "children"),
    Output("progress", "value"), 
    Output("progress", "label"),
    Output("totalsimtime", "children"),
    Output("risksimtime", "children"),
    Output("paramopttime", "children"),
    Output('positiontable', 'data'),  
    #secondtab
    Output('store-positiontable', 'data'),
    Output('positiontable', 'data_timestamp'),
    Output('timestamplist','data'),
    Output('VaR-noexpret', 'children'),
    Output('Var-expret-value', 'data'),
    Output('Es-noexpret', 'children'),
    Output('Es-expret-value', 'data'),
    Output('dcc_matrix', 'children'),
    Output('store-correlations','data'),
    Output('model-parameter-a', 'children'), 
    Output('model-parameter-b', 'children'),
    Output('model-parameter-1-a-b', 'children'),
    Output('univariateparams', 'children'),
    Output('store-ddofs', 'data'),
    ],
    [
    Input('start', 'n_clicks_timestamp'),
    Input('interval-component', 'n_intervals'),
    Input('stop', 'n_clicks_timestamp'),
    Input('reset', 'n_clicks_timestamp'),
    ],
    [
    State('start','children'),   
    State('bid_ask','value'),
    State('max_order_size','value'),
    State('order_num','value'),
    State('positiontable', 'data'), 
    State('positiontable', 'columns'),
    State('positiontable', 'data_timestamp'),
    State('positiontable','style_data_conditional'),
    State('timestamplist','data'),
    State('loading','value'),
    State('store-extractsamplebool','data'),
    State('date-picker','date'),
    State('margin','value'),
    State('store-theta','data'),    
    ],prevent_initial_call=True,
)
def updateData(
            #inputs
            start,
            n,
            stop,
            reset,
            #states
            startname,
            bidask,
            max_ordersize,
            order_num,
            data, 
            columns,
            data_timestamp,
            style_data_conditional_positiontable,
            data_timestamplist,
            loadingbar,
            extractsamplebool,
            dateselection,
            marginrequirement,
            lastheta
            ):
    
    #dashbugfix
    if len(dateselection)==19:
        dateselection=dateselection[:-9]

    marginrequirement=marginrequirement/100
    
    start=int(start)
    stop=int(stop)

    #time callback
    start_time_total = perf_counter()
    
    #callbacktrigger
    ctx = dash.callback_context
    input_id = ctx.triggered[0]["prop_id"].split(".")[0]    

    
    if input_id in 'start' and start==0:
        raise dash.exceptions.PreventUpdate
    
    #prevent callback if previous has not been completed
    if data_timestamplist==None or input_id in 'reset' or input_id in 'start':
        data_timestamplist=[0,1]
    else:
        values=data_timestamplist.values()
        data_timestamplist = list(values)[0]
        
    if data_timestamplist[-1]==data_timestamplist[-2]:  
        raise dash.exceptions.PreventUpdate

    #start,stop,reset logic

    if input_id in 'start':
        reset=0
        #start=1
    elif input_id in 'interval-component':
        stop=0
        reset=0
    elif input_id in 'stop':
        raise dash.exceptions.PreventUpdate
    else:
        pass
    
    #process selected sim parameters
    bid_ask_spread=(bidask/100)/2 #only 50% of bidask used in calculations 
    min_orderscycle,max_orderscycle=order_num

    #start with clean positiontable
    if (data == None) or data==[] or input_id in 'reset':
        df_positions = pd.DataFrame(columns=['Ticker', 'Price', 'Size','Notional','Buy-in','Trade Profit','Inv. Val.','P/L']).set_index('Ticker')
    else:
        df_positions=pd.DataFrame(data).set_index('Ticker')
    
    if start>0 and reset==0:

#----------------------------Generate Orders and Calc Returns--------------------------- 

        def extractsample(daysofhistoricaldata,date):
 
            datasample=pd.read_csv("prices.csv")
            datasample.set_index('date', inplace=True)
            sample_start=datasample[(datasample.index <= date)]
            sample_start_size=78*(daysofhistoricaldata)


            sample_start=sample_start.tail(sample_start_size)
            sample_start.to_csv('sample_start.csv')



            sample_sim=datasample[(datasample.index> date)] 
            sample_sim=sample_sim.head(78)
            sample_sim.to_csv('sample_sim.csv')

            return sample_start,sample_sim

        #avoid extracting initial sample when resume is pressed
        if extractsamplebool==None or 0: #empty store component triggerst sample start extraction, component filled after callback to avoid resampling
            sample_start,sample_sim=extractsample(daysofhistoricaldata, dateselection)     
        
        '''Sim Live Market Prices'''

        def simnewdata():
            sample_start= pd.read_csv('sample_start.csv',index_col=0)
            sample_sim= pd.read_csv('sample_sim.csv',index_col=0)

            simdata=sample_start.append(sample_sim.iloc[0]) #code generates new pricedata to a df designed to simulate pricedata
            sample_sim = sample_sim.iloc[1:] #drop the first row that was fed to sample_start

            simdata.to_csv('sample_start.csv') #overwrite sample_start with the new live trading data    
            sample_sim.to_csv('sample_sim.csv')

            return simdata

        simdata=simnewdata()
        currentdatetime=simdata.index[-1]+" EST"

        '''Trading Book'''

        def obtainlastprice(ordertickersymbol):
            data_order=pd.DataFrame()
            data_order['Price']=simdata.iloc[-1:][ordertickersymbol]
            data_order.reset_index(drop=True)
            orderticker=[ordertickersymbol]
            data_order=data_order.assign(Ticker=orderticker)
            data_order.reset_index(drop=True)
            return data_order

        def randorder():

            #random ticker with last price
            ordertickersymbol=random.choice(tickers)

            data_order=obtainlastprice(ordertickersymbol)


            #random order size
            order_notional_rand=np.random.randint(-max_ordersize, max_ordersize, size=1) #orders either negative (sell) or positive (buy)
            order_size_rand=np.round(order_notional_rand/data_order['Price']) #round so only whole numbers of shares are purchased and sold

            #appending to order dataframe
            data_order['Notional']=order_size_rand*data_order['Price']
            data_order['Size']=order_size_rand

            #compute order details based on spreads
            price_bid = data_order['Price']*(1-bid_ask_spread)
            price_ask = data_order['Price']*(1+bid_ask_spread)

            data_order['Buy-in'] = np.where(data_order['Size']>0, price_bid*data_order['Size'], price_ask*data_order['Size']) #bid or ask prices based on buying or selling
            data_order['Trade Profit']=data_order['Notional']-data_order['Buy-in']
            data_order = data_order.set_index(['Ticker'])

            return data_order, ordertickersymbol

        def updatepositiontable(data_order, ordertickersymbol, df_positions):

            #overwrite position size
            if ordertickersymbol in df_positions.index:
                df_positions=df_positions.add(data_order[['Size','Buy-in','Trade Profit']], fill_value=0)
            #add new position
            else:
                df_positions=df_positions.append(data_order[['Size','Buy-in','Trade Profit']])


            for ticker in df_positions.index.get_level_values(0):
                data_position=obtainlastprice(ticker)
                data_position=data_position.set_index(['Ticker'])
                df_positions.update(data_position['Price'])  

            #compute current position trading values
            df_positions['Notional']=df_positions['Size']*df_positions['Price']
            df_positions['Inv. Val.']=df_positions['Notional']-df_positions['Buy-in']-df_positions['Trade Profit']
            df_positions['P/L']=df_positions['Trade Profit']+df_positions['Inv. Val.']
            #reorder columns
            df_positions=df_positions[['Price', 'Size','Notional','Buy-in','Trade Profit','Inv. Val.','P/L']]

            return df_positions

        for numorders in range(random.randint(min_orderscycle,max_orderscycle)):
            data_order, ordertickersymbol=randorder()
            #orderhistory=orderhistory.append(data_order)
            df_positions=updatepositiontable(data_order, ordertickersymbol, df_positions)

        '''Deseasonalize'''
        #deseason rets of securities in the portfolio - df_positions
        def seasonality(prices):
            #compute returns
            rets=prices.pct_change().iloc[1: , :]

            #1.squared returns
            squared_rets=(rets**2)

            #2.mean squared returns per n intraday period
            for n in range(78):
                nthret=squared_rets.iloc[n::78, :]
                if n==0:
                    means=nthret.mean(axis=0).to_numpy()
                else:
                    means=np.vstack((means,nthret.mean(axis=0).to_numpy()))

            #3.Divide rets by seasonality term. These are standardised with mean 0 and stdev 1
            rets_deseason = rets.copy()
            for n in range(78):
                rets_deseason.iloc[n::78, :] /= np.sqrt(means[n])

            #4. Scale rets back to original rets
            rets_deseason_rescaled=rets.mean()+(rets_deseason-rets_deseason.mean())*(rets.std()/rets_deseason.std())
            
            return rets_deseason_rescaled
            
        rets=seasonality(simdata[list(df_positions.index.values)])
        
        #scale up for scipy optimize
        rets=rets*1000
     
        
#----------------------------Calculate Risk--------------------------- 

        '''DCC Forecasting'''

        #for dcc related code implementation, see:
        #https://github.com/tinoproductions/DirtyQuant/blob/master/DCC%20Model.ipynb
        #MIT License (MIT) 
        #Copyright (c) 2021 tinoproductions
            
        def DccForecast(df_positions, rets):

            numvars=len(list(df_positions.index.values)) #amount of variables/securities in portfolio. necessary for matrix operations
            #extracts the ower triangular matrix as a horizontal list
            def vecl(matrix):
                lower_matrix = np.tril(matrix,k=-1)
                array_with_zero=np.matrix(lower_matrix).A1

                array_without_zero = array_with_zero[array_with_zero!=0]
                return array_without_zero

            def loglike_norm_dcc_copula(theta, udata):
                N, T = np.shape(udata)
                llf = np.zeros((T,1))
                trdata = np.array(norm.ppf(udata).T, ndmin=2) #std. residuals (marginals of copula) are normally distributed (norm.ppf) for dependancy
                Rt, veclRt =  dcceq(theta,trdata)

                for i in range(0,T):
                    llf[i] = -0.5* np.log(np.linalg.det(Rt[:,:,i]))
                    llf[i] = llf[i] - 0.5 *  np.matmul(np.matmul(trdata[i,:] , (np.linalg.inv(Rt[:,:,i]) - np.eye(N))) ,trdata[i,:].T)
                llf = np.sum(llf)

                return -llf

            #lists needed after optimization
            lastQt=[]
            immediatedisturbances_list=[] #not initiating np array due to np.append generating full copys of data on each forloop iterration
            global dcceq
            def dcceq(theta,trdata):
                T, N = np.shape(trdata)
                a, b = theta
                if min(a,b)<0 or max(a,b)>1 or a+b > .999999:
                    a = .9999 - b
                Qt = np.zeros((N, N ,T))
                Qt[:,:,0] = np.cov(trdata.T)
                Rt =  np.zeros((N, N ,T))
                veclRt =  np.zeros((T, int(N*(N-1)/2)))
                Rt[:,:,0] = np.corrcoef(trdata.T)             
                
                for j in range(1,T):
                    global Qbar #use outside function for forecast
                    Qbar=Qt[:,:,0]
                    Qt[:,:,j] = Qbar * (1-a-b)
                    immediatedisturbance=np.matmul(trdata[[j-1]].T, trdata[[j-1]])                          
                    Qt[:,:,j] = Qt[:,:,j] + a * immediatedisturbance
                    Qt[:,:,j] = Qt[:,:,j] + b * Qt[:,:,j-1]
                    Rt[:,:,j] = np.divide(Qt[:,:,j] , np.matmul(np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2)).T , np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2))))    
                
                for j in range(0,T):
                    veclRt[j, :] = vecl(Rt[:,:,j].T)

                #capture last optimized qt value of timeseries
                Qt_end = Qt.flatten()
                Qt_end=Qt_end[(len(rets)-1)::len(rets)] #syntax: start:stop:step
                Qt_end=Qt_end.tolist()
                lastQt[0:(numvars**2)]=Qt_end #optimizer loops through function several times, only Qt vector of final and optimum outcome should be used for the forecast   

                return Rt, veclRt

            #computes uniformly distributed standardized residuals from 0 to 1 from returns and garch model parameters. t.cdf function
            global garch_t_to_u
            def garch_t_to_u(rets, res):
                mu = res.params['mu'] #mean return
                nu = res.params['nu'] #kurtosis
                est_residuals = rets - mu #estimated error term/residuals
                cond_vol = res.conditional_volatility #conditional volatility 
                std_residuals = est_residuals / np.sqrt(cond_vol) #standardized residuals
                udata = t.cdf(std_residuals, nu) #uniformly distributed cdf of standardized residuals
                
                return udata
   
            #this function takes returns, applys a garch model to it with a t distribution, then calls the function garch_t_to_u
            udata_list = []
            fcast_conditionalvariance_list = []
            univariatefit_list=[['omega'], ['alpha[1]'], ['beta[1]'],['nu']]
            univariatefit_df = pd.DataFrame(univariatefit_list, columns = ['Ticker'])
            ddofs_df = pd.DataFrame([['nu']], columns = ['Ticker'])
            
            global run_garch_on_return
            def run_garch_on_return(rets, udata_list):
                for x in rets: #forloop iterates through rets columns
                    am = arch_model(rets[x], dist = 't', p=1, q=1) #garch(1,1) model with t-distribution
                    res = am.fit(disp='off') #regression output of tickers in rets df
                    #coefficients for frontend
                    univariatefit_df[x] = [res.params[1].round(3), res.params[2].round(3), res.params[3].round(3), res.params[4].round(3)]
                    univariatefit_df["p"] = [res.pvalues[1].round(5), res.pvalues[2].round(5), res.pvalues[3].round(5), res.pvalues[4].round(5)]
                    univariatefit_df[x] = univariatefit_df[x].astype(str) + ' p:' + univariatefit_df["p"].astype(str)
                    univariatefit_df.drop('p', 1, inplace=True)
                    #ddof parameters for t-distributed mcm
                    ddofs_df[x] = [res.params[4].round(3)]

                    #uniformly distributed standardized residuals of return series
                    udata = garch_t_to_u(rets[x], res) #garch t_to_u_function call to change garch t volatility to uniform garch volatility
                    udata_list.append(udata)

                    #forecast variance for t+1 / needed for vector Dt in forecastingDCC function
                    fcastvariance =  res.forecast(horizon=1, reindex=False).variance
                    fcastvariance=fcastvariance['h.1'].iloc[0]
                    fcast_conditionalvariance_list.append(fcastvariance)

                return udata_list, fcast_conditionalvariance_list, univariatefit_df, ddofs_df      
            
            #1.Run Garch on Returns
            #2.Estimate dcc theta vals.
            #3.Forecast conditional variance of assets
            #4.Forecast conditional correlation
            #5.Derive forecasted conditional variance-covariance matrix
            #6.Calculate Value at Risk

            #1.Run Garch on Returns
            udata_list, fcast_conditionalvariance_list,univariatefit_df, ddofs_df = run_garch_on_return(rets.iloc[:,:numvars].dropna(), udata_list)
            
            #2.Estimate dcc theta vals.
            #optimizer constraints and bounds
            cons = ({'type': 'ineq', 'fun': lambda x:  -x[0]  -x[1] +1})
            bnds = ((0.0, 0.5), (0, 0.9997))
            #initialguess
            if lastheta==None:
                initialguess=[0.01,0.95]
            else:
                initialguess=lastheta
            
            #minimize loglikelihood
            optimizationtime = perf_counter()
            opt_out = minimize(loglike_norm_dcc_copula, np.array(initialguess), args = (udata_list,), bounds=bnds, constraints=cons)
            paramopttime='Param. Opt.: %s seconds' % round((perf_counter()-optimizationtime),2)
            
            #theta
            a,b=opt_out.x
            thetalist=[a,b]

            #capture dcc correlation process
            trdata = np.array(norm.ppf(udata_list).T, ndmin=2)
            Rt, veclRt = dcceq(opt_out.x, trdata)
            stock_names = [x.split()[0] for x in rets.columns]

            corr_name_list = []
            for i, name_a in enumerate(stock_names):
                if i == 0:
                    pass
                else:
                    for name_b in stock_names[:i]:
                        corr_name_list.append(name_a + "-" + name_b)            
            
            #save in df for graphing
            dcc_corr = pd.DataFrame(veclRt, columns= corr_name_list)
            dcc_corr['date']= rets.index

            dcc_corr=dcc_corr.to_dict('rows')
            

            #3.Forecast conditional variance of assets
            #variances for variance-covariance matrix-needed for VaR. Needed for Matrix Dt. Careful, Matrix Dt requires vol not variance
            Dt=np.sqrt(np.diag(np.array(fcast_conditionalvariance_list)))

            #4.Forecast conditional correlation, Rt
            listoflists=[]
            for i in range(0, len(lastQt), numvars):
                listoflists.append(lastQt[i:i+numvars])
            lastQt=np.array(listoflists)

            fcast_immediatedisturbances=np.zeros((numvars,numvars))
             
            Qtforecast=Qbar * (1-a-b) + a * fcast_immediatedisturbances + b * lastQt
            Qtforecast = np.reshape(Qtforecast, (numvars,numvars))
            Rtforecast = np.divide(Qtforecast , np.matmul(np.sqrt(np.array(np.diag(Qtforecast), ndmin=2)).T , np.sqrt(np.array(np.diag(Qtforecast), ndmin=2))))

            #5.Derive forecasted conditional variance-covariance matrix, Ht
            Htforecast=np.matmul(np.matmul(Dt, Rtforecast),Dt)

            return Rtforecast, Htforecast, a,b, dcc_corr,thetalist,paramopttime, univariatefit_df, ddofs_df
        
        #time callback
        start_time_risk = perf_counter()
        Rtforecast, Htforecast, a,b, dcc_corr,thetalist,paramopttime, univariatefit_df, ddofs_df=DccForecast(df_positions, rets)

        #lower dcc matrix for frontend
        def lowerdccfunc(Rtforecast,rets,absolute=False):
            dcc_matrix=pd.DataFrame(Rtforecast, index=list(rets.columns)).round(2)
            if absolute==True:
                dcc_matrix=dcc_matrix.abs()
            dcc_matrix.columns=list(rets.columns)
            lower_dcc_matrix=dcc_matrix.where(np.tril(np.ones(dcc_matrix.shape)).astype(bool))      
            #set tickers from index as row one for frontenddisplay:
            lower_dcc_matrix.reset_index(inplace=True)
            lower_dcc_matrix=lower_dcc_matrix.rename(columns={lower_dcc_matrix.columns[0]: ''})
            #changin columns+rows in callback are not possible, hence a workaround is to build dash datatable in callback and return to the html div in app layout
            return lower_dcc_matrix
        
        lower_dcc_matrix=lowerdccfunc(Rtforecast,rets)
        lower_dcc_matrix_abs=lowerdccfunc(Rtforecast,rets,absolute=True)

        #scale back after optimization
        rets=rets/1000
        Htforecast=Htforecast/(1000**2) #**2 due to variance scaled up by scaling factor squared
        
#--------------------------begin VaR---------------------        
        

        def portfolioVaR(df_positions,rets,covMatrix, ddofs_df):
            mc_num=5000
            meanrets=(rets.mean(axis=0)).to_numpy()
            meanrets=np.reshape(meanrets, (len(meanrets),1))
            covMatrix = np.array(covMatrix)
            ttoendofday=1 #only step ahead-VaR
            nstocks=df_positions.loc[:,'Size'].values

            bool_ind = nstocks < 0
            #increasing short position by marginrequirement
            nstocks = np.where(bool_ind ==True, nstocks*(1+marginrequirement), nstocks)
            
            lastprice=df_positions.loc[:,'Price'].values
            
            #determine weighted average of univariate ddofs form univariate garch fit
            weights_df=(df_positions['Notional'].abs())/(df_positions['Notional'].abs().sum())
            weights=weights_df.values
            ddofs=np.delete(ddofs_df.values[0],[0])

            #used subsequently for random draws
            ddof_weightedavg=np.round(np.sum(np.multiply(weights,ddofs)),3)
            
            portval=np.sum(np.multiply(lastprice,nstocks))
            portval=np.repeat(portval, mc_num)
            
            try: #still an error that covMatrix ocassionally is not positive-definitive...
        
                cholesky = np.linalg.cholesky(covMatrix)

                #loop not necessary but recovered from app script and not modified much
                counter=0
                for i in range(0, ttoendofday):
                    #draw random normal returns
                    draw=np.random.standard_t(ddof_weightedavg, mc_num) #averaage univariate ddof from portfolio
                    for x in nstocks[:-1]:#less one as first draw already completed
                        draw=np.vstack((draw, np.random.standard_t(ddof_weightedavg, mc_num)))
                    #correlated random returns
                    corrdraw=np.matmul(cholesky, draw)

                    #Add expected returns
                    corrdraw_plusexpret=np.add(meanrets, corrdraw)
                    corrdraw_plusexpret=1+corrdraw_plusexpret #for price multiplication

                    #price development of individual stocks
                    if counter==0:
                        stockpricechanges=(corrdraw_plusexpret.T * lastprice).T
                        lastprice=stockpricechanges

                    else:
                        stockpricechanges=np.multiply(lastprice,corrdraw_plusexpret)
                        lastprice=stockpricechanges

                    stockpricenotionals=(stockpricechanges.T * nstocks).T

                    portval=np.vstack((portval,stockpricenotionals.sum(axis=0)))

                    counter+=1


                confidencealpha=np.quantile(portval[1], alpha)
                lastportvalue=portval[0][0]
                
                #VaR
                ValueAtRisk=np.round(np.sqrt((confidencealpha-lastportvalue)**2),2)
                
                #ES
                belowvar_portvals=portval[1][portval[1]<confidencealpha]
                ES=np.round(np.sqrt((np.average(belowvar_portvals)-lastportvalue)**2),2)
                
                floatingprofit=df_positions['P/L'].sum()
                ValueAtRisk_flprofits=ValueAtRisk-floatingprofit
                ES_flprofits=ES-floatingprofit

                return ValueAtRisk, ValueAtRisk_flprofits, ES, ES_flprofits
            except:
                print('Covariance Matrix for MCM is not positive definitive')
                return 0.0, 0.0, 0.0, 0.0
            
        ValueAtRiskTotal_nofl, ValueAtRiskTotal_flprofits, ESTotal_nofl, ESTotal_flprofits=portfolioVaR(df_positions,rets,Htforecast, ddofs_df)
        
        def diveffect(Ht,df_positions,rets):
        
            df_diversification = pd.DataFrame(columns=['Ticker',
                                                       'Asset_noexpret',
                                                       'Asset_expret',
                                                       'Portfolio_less_asset_noexpret',
                                                       'Portfolio_less_asset_expret',
                                                       'Portfolio_noexpret',
                                                       'Portfolio_expret'
                                                      ])
            tickervariances=np.diag(Ht)
            iterration=0

            for ticker in list(df_positions.index.values):
                rets_iterration=rets.drop(ticker, axis=1)
                df_positions_iterration=df_positions.drop(index=ticker)
                ddofs_iterration=ddofs_df.drop(ticker, axis=1)
                #ticker ValueAtRisk
                if df_positions.loc[ticker]['Notional']>0:
                    ValueatRisk_ticker_noexpret=np.sqrt((z_score*df_positions.loc[ticker]['Notional']*np.sqrt(tickervariances[iterration]))**2)
                else:
                    ValueatRisk_ticker_noexpret=np.sqrt((z_score*df_positions.loc[ticker]['Notional']*(1+marginrequirement)*np.sqrt(tickervariances[iterration]))**2)
                iterration+1

                tickerloc=list(df_positions.index.values).index(ticker)


                #strip HT matrix for portfolio less one
                Ht_iterration=np.delete(Ht, (iterration-1),axis=0)
                Ht_iterration=np.delete(Ht_iterration, (iterration-1), axis=1)

                ValueAtRisk_lessticker_nofl,_,_,_=portfolioVaR(df_positions_iterration,rets_iterration, Ht_iterration, ddofs_iterration)

                df_diversification = df_diversification.append({'Ticker':ticker,
                                                                'Asset_noexpret':ValueatRisk_ticker_noexpret,
                                                                'Portfolio_less_asset_nofl':ValueAtRisk_lessticker_nofl,
                                                                'Portfolio_nofl':ValueAtRiskTotal_nofl}, ignore_index=True)
            #technically, the diversification benefit is not free of excpected returns as its is still included in the portfolioVaR function a tthe moment 
            
            df_diversification['Div. Benefit_noexpret']=(((df_diversification['Asset_noexpret']+df_diversification['Portfolio_less_asset_nofl'])-df_diversification['Portfolio_nofl'])/ValueAtRiskTotal_nofl).round(2)
            df_diversification=df_diversification.set_index('Ticker')
            
            return df_diversification
        
        df_diversification=diveffect(Htforecast,df_positions,rets)
        
        df_positions['Div. Benefit_noexpret']= df_diversification['Div. Benefit_noexpret']
        df_positions['Div. Effect']=df_positions['Div. Benefit_noexpret']
        
        ValueAtRiskTotal_nofl=np.round(ValueAtRiskTotal_nofl,2)
        ValueAtRiskTotal_flprofits=np.round(ValueAtRiskTotal_flprofits,2)
        ESTotal_nofl=np.round(ESTotal_nofl,2)
        ESTotal_flprofits=np.round(ESTotal_flprofits,2)
 
        risksimtime='Risk: %s seconds' % round((perf_counter() - start_time_risk),2)  

        
#----------------------------DesignOutput---------------------------        
        
        #following styling functions are taken from dash plotly documentation with some modifications
        #https://dash.plotly.com/datatable/conditional-formatting
        def discrete_background_color_bins(lower_dcc_matrix, n_bins=5, columns='all'):
            
            bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
            if columns == 'all':
                if 'id' in lower_dcc_matrix:
                    df_numeric_columns = lower_dcc_matrix.select_dtypes('number').drop(['id'], axis=1)
                else:
                    df_numeric_columns = lower_dcc_matrix.select_dtypes('number')
            else:
                df_numeric_columns = lower_dcc_matrix[columns]
            df_max = df_numeric_columns.max().max()
            df_min = df_numeric_columns.min().min()
            ranges = [
                ((df_max - df_min) * i) + df_min
                for i in bounds
            ]
            styles = []
            legend = []
            for i in range(1, len(bounds)):
                min_bound = ranges[i - 1]
                max_bound = ranges[i]
                backgroundColor = colorlover.scales[str(n_bins)]['seq']['Blues'][i - 1]
                color = 'white' if i > len(bounds) / 2. else 'inherit'
                
                for column in df_numeric_columns:
                    styles.append({
                        'if': {
                            'filter_query': (
                                '{{{column}}} >= {min_bound}' +
                                (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                            ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                            'column_id': column
                        },
                        'backgroundColor': backgroundColor,
                        'color': color
                    })
                    
                styles.append({'if': {
                            'column_id': ''
                        },
                        'fontWeight':'bold','backgroundColor': 'rgb(248,248,248)'
                })
                styles.append({
                    "if": {"state": "active"},
                    "backgroundColor": "rgba(150, 180, 225, 0.2)",
                    "border": "1px solid blue",
                })
                styles.append({
                    "if": {"state": "selected"},
                    "backgroundColor": "rgba(0, 116, 217, .03)",
                    "border": "1px solid blue",
                })
                

                legend.append(
                    html.Div(style={'display': 'inline-block', 'width': '60px'}, children=[
                        html.Div(
                            style={
                                'backgroundColor': backgroundColor,
                                'borderLeft': '1px rgb(50, 50, 50) solid',
                                'height': '10px'
                            }
                        ),
                        html.Small(round(min_bound, 2), style={'paddingLeft': '2px'})
                    ])
                )

            return (styles, html.Div(legend, style={'padding': '5px 0 5px 0'}))


        (styles, legend) = discrete_background_color_bins(lower_dcc_matrix_abs)#colorcode absolute correlations        
        
        dcc_matrix_htmldiv = html.Div(
                            [legend,
                                dash_table.DataTable(
                                    id='inner_matrix_div',
                                    data=lower_dcc_matrix.to_dict('rows'),
                                    columns=[
                                        {'id': i, 'name': i} for i in lower_dcc_matrix.columns
                                    ],
                                    style_data_conditional=styles,
                                    selected_columns=[],
                                    selected_rows=[],
                                    page_action="native",
                                    style_header={'fontWeight':'bold','backgroundColor': 'rgb(248,248,248)'},
                                    style_cell={'fontSize':15, 'font-family':'Helvetica', 'textAlign': 'center'},
                                    style_table={'overflowX': 'auto', 'overflowY': 'auto'},
                                     ),
                                    
                            ]
                        )

        global dashpositiontable       
        def dashpositiontable(df_positions):

            #assign dedicated front-end df
            df_positions_dash=df_positions.copy(deep=True)

            #reset index for front-end display
            df_positions_dash.reset_index(inplace=True)

            #resort columns
            df_positions_dash = df_positions_dash[['Ticker','Price', 'Size','Notional','Buy-in','Trade Profit','Inv. Val.','P/L','Div. Effect']]

            df_positions_dash_values=df_positions_dash.loc[:, df_positions_dash.columns != 'Ticker']
            df_positions_dash_values=df_positions_dash_values.astype(float).round(2)
            df_positions_dash_string=df_positions_dash.loc[:, df_positions_dash.columns == 'Ticker']
            df_positions_dash=pd.concat([df_positions_dash_string, df_positions_dash_values], axis=1, join="inner")
 
            return df_positions_dash

        dashpositiontable_df=dashpositiontable(df_positions)

        #styling dash positiontable
        def data_bars(df, column):
            n_bins = 100
            bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
            ranges = [
                ((1 - 0) * i) + df[column].min()
                for i in bounds
            ]
            styles = []
            for i in range(1, len(bounds)):
                min_bound = ranges[i - 1]
                max_bound = ranges[i]
                max_bound_percentage = bounds[i] * 100
                styles.append({
                    'if': {
                        'filter_query': (
                            '{{{column}}} >= {min_bound}' +
                            (' && {{{column}}} < {max_bound}' if (i < len(bounds) - 1) else '')
                        ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                        'column_id': column
                    },
                    'background': (
                        """
                            linear-gradient(90deg,
                            #7fbbff 0%,
                            #7fbbff {max_bound_percentage}%,
                            white {max_bound_percentage}%,
                            white 100%)
                        """.format(max_bound_percentage=max_bound_percentage)
                    ),
                    'paddingBottom': 2,
                    'paddingTop': 2
                },
                )

            return styles        

        styles2= data_bars(dashpositiontable_df,'Div. Effect')
        
        style_pl=[
            {
                'if': {
                    'filter_query': '{P/L} >=0',
                    'column_id': 'P/L'
                },
                'backgroundColor': '#9cf986',
            },
            {
                'if': {
                    'filter_query': '{P/L} <0',
                    'column_id': 'P/L'
                },
                'backgroundColor': '#f46e6e',
            }
            ]
        
        style_data_conditional_positiontable=style_data_conditional_positiontable+styles2+style_pl
        
        univariatefit_htmldiv = html.Div(
                            [html.H5("Univariate GARCH Parameters"),
                                dash_table.DataTable(
                                    id='univariateparameters_div',
                                    data=univariatefit_df.to_dict('rows'),
                                    columns=[
                                        {'id': i, 'name': i} for i in univariatefit_df.columns
                                    ],
                                    style_data_conditional=style_data_conditional_positiontable,
                                    selected_columns=[],
                                    selected_rows=[],
                                    page_action="native",
                                    style_header={'fontWeight':'bold','backgroundColor': 'rgb(248,248,248)'},
                                    style_cell={'fontSize':15, 'font-family':'Helvetica', 'textAlign': 'center'},
                                    style_table={'overflowX': 'auto', 'overflowY': 'auto'},
                                     ),
                                    
                            ]
                        )
        dashpositiontable_df = dashpositiontable_df.to_dict('rows')
        datatabletimestamp=int(perf_counter())
        data_timestamplist.append(datatabletimestamp)     
        data_timestamplist=dict.fromkeys("T",data_timestamplist)
        totalsimtime='Total: %s seconds' % round((perf_counter() - start_time_total),2)
        
        #progress bar
        #78data points for 1 day simulation
        #first run is n==0
        progress = round((((n+1)/78)*100),1)
        if progress>100:#avoid greater percentage if last calc not completed and reinitiated
            progress=100.0

        rets=rets.to_dict('rows')
        ddofs_df=ddofs_df.to_dict('rows')
        
        
        #currentdate time passed once as a dummy variable for dcc store component 
            #tab1 output.tab2 output.stored tab1 ouptut funivor return 
        return thetalist,marginrequirement,Htforecast,rets,currentdatetime,currentdatetime,[{"label": "Incl. Fl. Profits", "value": 1, "disabled": False},],style_data_conditional_positiontable ,style_data_conditional_positiontable ,loadingbar,progress,f"{progress} %" if progress >= 8 else "", totalsimtime,risksimtime,paramopttime, dashpositiontable_df,dashpositiontable_df,datatabletimestamp,data_timestamplist, ValueAtRiskTotal_nofl,ValueAtRiskTotal_flprofits,ESTotal_nofl, ESTotal_flprofits, dcc_matrix_htmldiv,dcc_corr, 'a: %s' % np.round(a,5),'b: %s' % np.round(b,5),'1-a-b: %s' % np.round((1-a-b),5),univariatefit_htmldiv, ddofs_df      
    #if resetted    
    else:
        df_positions = pd.DataFrame(columns=['Ticker', 'Price', 'Size','Notional','Buy-in','Trade Profit','Inv. Val.','P/L','Div. Effect']).set_index('Ticker')
        dashpositiontable_df=dashpositiontable(df_positions)
        dashpositiontable_df = dashpositiontable_df.to_dict('rows')
        
        dcc_corr = pd.DataFrame()
        dcc_corr=dcc_corr.to_dict('rows')
        
        ddofs_df = pd.DataFrame()
        ddofs_df=ddofs_df.to_dict('rows')
        
        dcc_matrix_htmldiv = html.Div()
        univariatefit_htmldiv = html.Div()
    
        progress=0
        currentdatetime=""
        rets=0
        Htforecast=0
        thetalist=[0.01,0.95]
        
        datatabletimestamp=int(perf_counter())
        return thetalist,marginrequirement,Htforecast,rets,currentdatetime,currentdatetime,[{"label": "Incl. Exp. Returns", "value": 1, "disabled": True}],style_data_conditional_positiontable,style_data_conditional_positiontable,loadingbar,0,"","","","",dashpositiontable_df,dashpositiontable_df,datatabletimestamp,dict.fromkeys("T",[0]),"","","","",dcc_matrix_htmldiv,dcc_corr,'a:','b:','1-a-b:',univariatefit_htmldiv, ddofs_df
        

if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
